# FreshHarvest Hyperparameter Tuning

This notebook demonstrates hyperparameter tuning for the FreshHarvest fruit freshness classification model using various optimization techniques.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add src to path
sys.path.append('../src')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Hyperparameter optimization libraries
try:
    import optuna
    OPTUNA_AVAILABLE = True
except ImportError:
    print("Optuna not available. Install with: pip install optuna")
    OPTUNA_AVAILABLE = False

try:
    from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
    from scikeras.wrappers import KerasClassifier
    SKLEARN_AVAILABLE = True
except ImportError:
    print("SciKeras not available. Install with: pip install scikeras")
    SKLEARN_AVAILABLE = False

from cvProject_FreshHarvest.utils.common import read_yaml, setup_logging
from cvProject_FreshHarvest.models.cnn_models import FreshHarvestCNN

# Setup
setup_logging()
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

Optuna not available. Install with: pip install optuna
SciKeras not available. Install with: pip install scikeras
TensorFlow version: 2.19.0
GPU available: []


## 1. Load Configuration and Data

In [2]:
# Load configuration
config = read_yaml('../config/config.yaml')
print("Configuration loaded:")
print(f"- Image size: {config['data']['image_size']}")
print(f"- Number of classes: {config['data']['num_classes']}")
print(f"- Batch size: {config['training']['batch_size']}")
print(f"- Learning rate: {config['training']['learning_rate']}")

Configuration loaded:
- Image size: [224, 224]
- Number of classes: 16
- Batch size: 32
- Learning rate: 0.001


In [3]:
# Create data generators
def create_data_generators(batch_size=32, validation_split=0.2):
    """Create training and validation data generators."""
    
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )
    
    # Validation data generator (no augmentation)
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    # Create generators
    train_generator = train_datagen.flow_from_directory(
        '../data/processed/train',
        target_size=tuple(config['data']['image_size']),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )
    
    val_generator = val_datagen.flow_from_directory(
        '../data/processed/val',
        target_size=tuple(config['data']['image_size']),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, val_generator

# Create initial generators
train_gen, val_gen = create_data_generators()
print(f"Training samples: {train_gen.samples}")
print(f"Validation samples: {val_gen.samples}")
print(f"Classes: {list(train_gen.class_indices.keys())}")

Found 11199 images belonging to 16 classes.
Found 3200 images belonging to 16 classes.
Training samples: 11199
Validation samples: 3200
Classes: ['F_Banana', 'F_Lemon', 'F_Lulo', 'F_Mango', 'F_Orange', 'F_Strawberry', 'F_Tamarillo', 'F_Tomato', 'S_Banana', 'S_Lemon', 'S_Lulo', 'S_Mango', 'S_Orange', 'S_Strawberry', 'S_Tamarillo', 'S_Tomato']


## 2. Define Hyperparameter Search Space

In [4]:
# Define hyperparameter search space
HYPERPARAMETER_SPACE = {
    # Model architecture
    'model_type': ['basic', 'improved', 'lightweight'],
    
    # Training parameters
    'learning_rate': [0.0001, 0.0005, 0.001, 0.005, 0.01],
    'batch_size': [16, 32, 64],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    
    # Regularization
    'dropout_rate': [0.2, 0.3, 0.4, 0.5],
    'l2_regularization': [0.0001, 0.001, 0.01],
    
    # Data augmentation
    'rotation_range': [10, 20, 30],
    'zoom_range': [0.05, 0.1, 0.15],
    'brightness_range': [0.1, 0.2, 0.3],
    
    # Architecture specific
    'dense_units': [64, 128, 256],
    'conv_filters': [32, 64, 128]
}

print("Hyperparameter search space defined:")
for param, values in HYPERPARAMETER_SPACE.items():
    print(f"- {param}: {values}")

Hyperparameter search space defined:
- model_type: ['basic', 'improved', 'lightweight']
- learning_rate: [0.0001, 0.0005, 0.001, 0.005, 0.01]
- batch_size: [16, 32, 64]
- optimizer: ['adam', 'rmsprop', 'sgd']
- dropout_rate: [0.2, 0.3, 0.4, 0.5]
- l2_regularization: [0.0001, 0.001, 0.01]
- rotation_range: [10, 20, 30]
- zoom_range: [0.05, 0.1, 0.15]
- brightness_range: [0.1, 0.2, 0.3]
- dense_units: [64, 128, 256]
- conv_filters: [32, 64, 128]


## 3. Manual Grid Search

In [5]:
def create_model_with_params(learning_rate=0.001, dropout_rate=0.3, dense_units=128, model_type='lightweight'):
    """Create model with specified hyperparameters."""
    
    # Initialize CNN model
    cnn_model = FreshHarvestCNN('../config/config.yaml')
    
    # Create model based on type
    if model_type == 'basic':
        model = cnn_model.create_basic_cnn()
    elif model_type == 'improved':
        model = cnn_model.create_improved_cnn()
    else:
        model = cnn_model.create_lightweight_cnn()
    
    # Compile with specified parameters
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    return model

# Test model creation
test_model = create_model_with_params()
print(f"Test model created with {test_model.count_params()} parameters")

2025-06-19 07:50:20,833 - root - INFO - CNN model initialized
2025-06-19 07:50:21,262 - root - INFO - Created lightweight CNN model with 83019 parameters
Test model created with 83019 parameters


In [6]:
def manual_grid_search(param_combinations, max_trials=5):
    """Perform manual grid search over hyperparameters."""
    
    results = []
    
    for i, params in enumerate(param_combinations[:max_trials]):
        print(f"\nTrial {i+1}/{min(len(param_combinations), max_trials)}")
        print(f"Parameters: {params}")
        
        try:
            # Create model
            model = create_model_with_params(**params)
            
            # Create data generators with batch size
            batch_size = params.get('batch_size', 32)
            train_gen, val_gen = create_data_generators(batch_size=batch_size)
            
            # Create callbacks
            callbacks = [
                EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)
            ]
            
            # Train model (short training for hyperparameter search)
            history = model.fit(
                train_gen,
                epochs=5,  # Short training for hyperparameter search
                validation_data=val_gen,
                callbacks=callbacks,
                verbose=1,
                steps_per_epoch=min(50, train_gen.samples // batch_size),
                validation_steps=min(20, val_gen.samples // batch_size)
            )
            
            # Record results
            best_val_acc = max(history.history['val_accuracy'])
            best_val_loss = min(history.history['val_loss'])
            
            result = {
                'trial': i + 1,
                'params': params,
                'val_accuracy': best_val_acc,
                'val_loss': best_val_loss,
                'model_params': model.count_params()
            }
            
            results.append(result)
            print(f"Best validation accuracy: {best_val_acc:.4f}")
            
        except Exception as e:
            print(f"Trial failed: {e}")
            continue
    
    return results

# Define a few parameter combinations to test
test_combinations = [
    {'learning_rate': 0.001, 'batch_size': 32, 'model_type': 'lightweight'},
    {'learning_rate': 0.0005, 'batch_size': 32, 'model_type': 'lightweight'},
    {'learning_rate': 0.001, 'batch_size': 16, 'model_type': 'lightweight'},
    {'learning_rate': 0.001, 'batch_size': 32, 'model_type': 'basic'},
    {'learning_rate': 0.005, 'batch_size': 32, 'model_type': 'lightweight'}
]

print(f"Starting manual grid search with {len(test_combinations)} combinations...")

Starting manual grid search with 5 combinations...


In [7]:
# Run manual grid search
if len(test_combinations) > 0:
    print("\n🚀 Starting Manual Grid Search...")
    results = manual_grid_search(test_combinations, max_trials=3)
    
    if results:
        # Convert to DataFrame for analysis
        results_df = pd.DataFrame(results)
        print("\n📊 Grid Search Results:")
        print(results_df[['trial', 'val_accuracy', 'val_loss', 'model_params']])
        
        # Find best configuration
        best_idx = results_df['val_accuracy'].idxmax()
        best_result = results_df.iloc[best_idx]
        
        print(f"\n🏆 Best Configuration:")
        print(f"   Trial: {best_result['trial']}")
        print(f"   Validation Accuracy: {best_result['val_accuracy']:.4f}")
        print(f"   Validation Loss: {best_result['val_loss']:.4f}")
        print(f"   Parameters: {best_result['params']}")
    else:
        print("❌ No successful trials in grid search")
else:
    print("⚠️ No test combinations defined")


🚀 Starting Manual Grid Search...

Trial 1/3
Parameters: {'learning_rate': 0.001, 'batch_size': 32, 'model_type': 'lightweight'}
Trial failed: create_model_with_params() got an unexpected keyword argument 'batch_size'

Trial 2/3
Parameters: {'learning_rate': 0.0005, 'batch_size': 32, 'model_type': 'lightweight'}
Trial failed: create_model_with_params() got an unexpected keyword argument 'batch_size'

Trial 3/3
Parameters: {'learning_rate': 0.001, 'batch_size': 16, 'model_type': 'lightweight'}
Trial failed: create_model_with_params() got an unexpected keyword argument 'batch_size'
❌ No successful trials in grid search


## 4. Optuna Hyperparameter Optimization

In [8]:
# Optuna optimization (if available)
if OPTUNA_AVAILABLE:
    print("🔬 Setting up Optuna optimization...")
    
    def objective(trial):
        """Optuna objective function."""
        
        # Suggest hyperparameters
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
        batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
        dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
        model_type = trial.suggest_categorical('model_type', ['basic', 'lightweight'])
        
        try:
            # Create model with suggested parameters
            model = create_model_with_params(
                learning_rate=learning_rate,
                dropout_rate=dropout_rate,
                model_type=model_type
            )
            
            # Create data generators
            train_gen, val_gen = create_data_generators(batch_size=batch_size)
            
            # Train model
            history = model.fit(
                train_gen,
                epochs=3,  # Short training for optimization
                validation_data=val_gen,
                verbose=0,
                steps_per_epoch=min(20, train_gen.samples // batch_size),
                validation_steps=min(10, val_gen.samples // batch_size)
            )
            
            # Return best validation accuracy
            return max(history.history['val_accuracy'])
            
        except Exception as e:
            print(f"Trial failed: {e}")
            return 0.0
    
    # Create study and optimize
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=5, timeout=1800)  # 30 minutes max
    
    print("\n🏆 Optuna Optimization Results:")
    print(f"Best trial: {study.best_trial.number}")
    print(f"Best value: {study.best_value:.4f}")
    print(f"Best params: {study.best_params}")
    
    # Plot optimization history
    try:
        fig = optuna.visualization.plot_optimization_history(study)
        fig.show()
        
        fig = optuna.visualization.plot_param_importances(study)
        fig.show()
    except Exception as e:
        print(f"Visualization failed: {e}")
        
else:
    print("⚠️ Optuna not available. Install with: pip install optuna")

⚠️ Optuna not available. Install with: pip install optuna


## 5. Results Analysis and Visualization

In [9]:
# Analyze hyperparameter importance
def analyze_hyperparameter_importance(results):
    """Analyze which hyperparameters have the most impact."""
    
    if not results:
        print("No results to analyze")
        return
    
    # Extract hyperparameters and performance
    param_data = []
    for result in results:
        params = result['params'].copy()
        params['val_accuracy'] = result['val_accuracy']
        param_data.append(params)
    
    df = pd.DataFrame(param_data)
    
    # Correlation analysis
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 1:
        plt.figure(figsize=(10, 8))
        correlation_matrix = df[numeric_cols].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
        plt.title('Hyperparameter Correlation Matrix')
        plt.tight_layout()
        plt.show()
    
    # Performance by categorical parameters
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    if len(categorical_cols) > 0:
        fig, axes = plt.subplots(1, len(categorical_cols), figsize=(5*len(categorical_cols), 5))
        if len(categorical_cols) == 1:
            axes = [axes]
        
        for i, col in enumerate(categorical_cols):
            if col != 'val_accuracy':
                df.boxplot(column='val_accuracy', by=col, ax=axes[i])
                axes[i].set_title(f'Validation Accuracy by {col}')
                axes[i].set_xlabel(col)
        
        plt.tight_layout()
        plt.show()

# Analyze results if available
if 'results' in locals() and results:
    analyze_hyperparameter_importance(results)
else:
    print("No hyperparameter tuning results to analyze")

No hyperparameter tuning results to analyze


## 6. Best Configuration Training

In [10]:
# Train final model with best configuration
def train_best_model(best_params, epochs=20):
    """Train model with best hyperparameters for longer."""
    
    print(f"\n🚀 Training final model with best parameters: {best_params}")
    
    # Create model with best parameters
    model = create_model_with_params(**best_params)
    
    # Create data generators
    batch_size = best_params.get('batch_size', 32)
    train_gen, val_gen = create_data_generators(batch_size=batch_size)
    
    # Enhanced callbacks for final training
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            filepath='../models/best_hypertuned_model.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]
    
    # Train model
    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=callbacks,
        verbose=1
    )
    
    return model, history

# Train best model if we have results
if 'best_result' in locals():
    best_model, best_history = train_best_model(best_result['params'])
    
    # Plot final training history
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(best_history.history['accuracy'], label='Training Accuracy')
    plt.plot(best_history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(best_history.history['loss'], label='Training Loss')
    plt.plot(best_history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Best model saved to models/best_hypertuned_model.h5")
    print(f"Final validation accuracy: {max(best_history.history['val_accuracy']):.4f}")
else:
    print("No best configuration found to train final model")

No best configuration found to train final model


## 7. Summary and Recommendations

In [11]:
print("\n" + "="*80)
print("HYPERPARAMETER TUNING SUMMARY")
print("="*80)

print("\n🔍 SEARCH SPACE EXPLORED:")
for param, values in HYPERPARAMETER_SPACE.items():
    print(f"  • {param}: {values}")

if 'results' in locals() and results:
    print(f"\n📊 TRIALS COMPLETED: {len(results)}")
    
    if 'best_result' in locals():
        print(f"\n🏆 BEST CONFIGURATION:")
        print(f"  • Validation Accuracy: {best_result['val_accuracy']:.4f}")
        print(f"  • Parameters: {best_result['params']}")
        
        print(f"\n💡 KEY INSIGHTS:")
        best_params = best_result['params']
        
        if 'learning_rate' in best_params:
            lr = best_params['learning_rate']
            if lr < 0.001:
                print(f"  • Lower learning rate ({lr}) performed best - suggests need for careful optimization")
            elif lr > 0.005:
                print(f"  • Higher learning rate ({lr}) performed best - model can handle aggressive training")
            else:
                print(f"  • Moderate learning rate ({lr}) performed best - balanced approach")
        
        if 'model_type' in best_params:
            model_type = best_params['model_type']
            print(f"  • {model_type.capitalize()} architecture performed best")
        
        if 'batch_size' in best_params:
            batch_size = best_params['batch_size']
            if batch_size <= 16:
                print(f"  • Smaller batch size ({batch_size}) preferred - suggests noisy gradients help")
            else:
                print(f"  • Larger batch size ({batch_size}) preferred - stable gradient updates")

print("\n🚀 NEXT STEPS:")
print("  1. Run longer training with best hyperparameters")
print("  2. Implement advanced techniques (learning rate scheduling, warm restarts)")
print("  3. Try ensemble methods with top configurations")
print("  4. Explore transfer learning with pre-trained models")
print("  5. Optimize for deployment (quantization, pruning)")

print("\n📝 RECOMMENDATIONS:")
print("  • Use automated hyperparameter optimization for production models")
print("  • Consider multi-objective optimization (accuracy vs speed vs size)")
print("  • Implement cross-validation for more robust hyperparameter selection")
print("  • Monitor hyperparameter sensitivity for model stability")


HYPERPARAMETER TUNING SUMMARY

🔍 SEARCH SPACE EXPLORED:
  • model_type: ['basic', 'improved', 'lightweight']
  • learning_rate: [0.0001, 0.0005, 0.001, 0.005, 0.01]
  • batch_size: [16, 32, 64]
  • optimizer: ['adam', 'rmsprop', 'sgd']
  • dropout_rate: [0.2, 0.3, 0.4, 0.5]
  • l2_regularization: [0.0001, 0.001, 0.01]
  • rotation_range: [10, 20, 30]
  • zoom_range: [0.05, 0.1, 0.15]
  • brightness_range: [0.1, 0.2, 0.3]
  • dense_units: [64, 128, 256]
  • conv_filters: [32, 64, 128]

🚀 NEXT STEPS:
  1. Run longer training with best hyperparameters
  2. Implement advanced techniques (learning rate scheduling, warm restarts)
  3. Try ensemble methods with top configurations
  4. Explore transfer learning with pre-trained models
  5. Optimize for deployment (quantization, pruning)

📝 RECOMMENDATIONS:
  • Use automated hyperparameter optimization for production models
  • Consider multi-objective optimization (accuracy vs speed vs size)
  • Implement cross-validation for more robust hype